In [1]:
%load_ext autoreload
%autoreload 2

import torch
from model.las_model import Listener, Speller, LAS

PATH = 'C:/Users/USER/Desktop/jiwon/las-pytorch/runs/kospeech_dropout0.3_retry-BEST_LOSS-epoch16.pth.tar'
checkpoint = torch.load(PATH)
print(checkpoint['etype'])
listener = Listener(input_feature_dim= checkpoint['einput'],
        hidden_size=checkpoint['ehidden'],
        num_layers=checkpoint['elayer'],
        dropout=checkpoint['edropout'],
        bidirectional=True,
        rnn_unit="LSTM",
        use_gpu=True)

speller = Speller(hidden_size= checkpoint['dhidden'],
        num_layers= checkpoint['dlayer'],
        bidirectional= True,
        rnn_unit= "LSTM",
        vocab_size= checkpoint['dvocab_size'],                       # 61 phonemes + 2 for <sos> & <eos>
        multi_head= 1,                          # Number of heads for multi-head attention
        decode_mode= 1,                         # Decoding mode, 0 : feed char distribution to next timestep, 1: feed argmax, 2: feed sampled vector
        use_mlp_in_attention= True,                  # Set to False to exclude phi and psi in attention formula
        mlp_dim_in_attention= 64,                   #
        mlp_activate_in_attention= 'relu',        #
        listener_hidden_size= 512,
        max_label_len= 576)
        

las_model = LAS(listener, speller)
las_model.cuda()
optimizer = torch.optim.Adam(params=las_model.parameters(), lr=0.002)


las_model.load_state_dict(checkpoint['state_dict'])
optimizer.load_state_dict(checkpoint['optim_dict'])
epoch = checkpoint['epoch']


las_model.eval()
#model.eval()
#print(model)

<class 'torch.nn.modules.rnn.LSTM'>


C:\Users\USER\anaconda3\envs\kospeech\lib\site-packages\torch\nn\modules\rnn.py:62: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.3 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


LAS(
  (listener): Listener(
    (pLSTM_layer0): pBLSTMLayer(
      (BLSTM): LSTM(80, 512, batch_first=True, dropout=0.3, bidirectional=True)
    )
    (pLSTM_layer1): pBLSTMLayer(
      (BLSTM): LSTM(2048, 512, batch_first=True, dropout=0.3, bidirectional=True)
    )
    (pLSTM_layer2): pBLSTMLayer(
      (BLSTM): LSTM(2048, 512, batch_first=True, dropout=0.3, bidirectional=True)
    )
  )
  (speller): Speller(
    (rnn_layer): LSTM(2524, 1024, num_layers=2, batch_first=True)
    (attention): Attention(
      (softmax): Softmax(dim=-1)
      (phi): Linear(in_features=1024, out_features=64, bias=True)
      (psi): Linear(in_features=1024, out_features=64, bias=True)
    )
    (character_distribution): Linear(in_features=2048, out_features=1500, bias=True)
    (softmax): LogSoftmax(dim=-1)
  )
)

In [13]:
checkpoint['dvocab_size']

30

idx,input,label



In [8]:
test_loader.shape

AttributeError: 'DataLoader' object has no attribute 'shape'

In [11]:
from tqdm.auto import tqdm
from solver.solver import batch_iterator

from utils.data import AudioDataLoader, AudioDataset
import numpy as np

params={"data":{"vocab":"C:/Users/USER/Desktop/jiwon/las-pytorch/data/kospeech/processed/idx2chap.csv","batch_size":8,"test":"C:/Users/USER/Desktop/jiwon/las-pytorch/data/kospeech/processed/test-clean-test.csv","vocab_size":1500},"model":{"listener":{"num_layers":3}}}
test_dataset = AudioDataset(params, "test")
test_loader = AudioDataLoader(test_dataset, num_workers=1).loader


tf_rate = 0.9
val_loss = []
val_ler = []
val_step = 0
for i, (data) in tqdm(enumerate(test_loader), total=len(test_loader), leave=False, desc="Test"):
    # print(
    #     f"Current Epoch: {epoch} | Epoch step: {epoch_step}/{len(train_loader)} Validating step: {val_step}/{len(dev_loader)}",
    #     end="\r",
    #     flush=True,
    # )

    inputs = data[1]["inputs"].cuda()
    labels = data[2]["targets"].cuda()

    batch_loss, batch_ler = batch_iterator(
        batch_data=inputs,
        batch_label=labels,
        las_model=las_model,
        optimizer=optimizer,
        tf_rate=tf_rate,
        is_training=False,
        max_label_len=30,
        label_smoothing=0.1,
        vocab_dict=test_dataset.char2idx,
    )
    if i % 100 == 0:
        torch.cuda.empty_cache()
    val_loss.append(batch_loss)
    val_ler.extend(batch_ler)
    val_step += 1 

val_loss = np.array([sum(val_loss) / len(val_loss)])
val_ler = np.array([sum(val_ler) / len(val_ler)])

Test:   0%|          | 0/8 [00:00<?, ?it/s]

pred : [array(['이', '', '대', '리', '', '아', '니', '야', '<sos>', '<sos>', '<sos>',
       '<sos>', '<sos>', '<sos>', '<sos>'], dtype='<U5'), array(['안', '녕', '하', '세', '요', '', '일', '찍', '', '출', '근', '하', '시', '네',
       '요'], dtype='<U1')]
true : [array(['이', '', '대', '리', '', '아', '니', '야', '<sos>', '<sos>', '<sos>',
       '<sos>', '<sos>', '<sos>', '<sos>'], dtype='<U5'), array(['안', '녕', '하', '세', '요', '', '일', '찍', '', '출', '근', '하', '시', '네',
       '요'], dtype='<U1')]
pred : [array(['만', '', '원', '지', '하', '철', '에', '', '타', '면', '', '한', '', '',
       '', '', '', '비', '비', '해', '서', '', '일', '', '', '못', '', '하', '겠',
       '더'], dtype='<U1'), array(['나', '도', '요', '<sos>', '<sos>', '<sos>', '<sos>', '<sos>',
       '<sos>', '<sos>', '<sos>', '<sos>', '<sos>', '<sos>', '<sos>',
       '<sos>', '<sos>', '<sos>', '<sos>', '<sos>', '<sos>', '<sos>',
       '<sos>', '<sos>', '<sos>', '<sos>', '<sos>', '<sos>', '<sos>',
       '<sos>'], dtype='<U5')]
true : [array(['만', '원', '', '

In [13]:
from tqdm.auto import tqdm
from solver.solver import batch_iterator

from utils.data import AudioDataLoader, AudioDataset
import numpy as np

params={"data":{"vocab":"C:/Users/USER/Desktop/jiwon/las-pytorch/data/kospeech/processed/idx2chap.csv","batch_size":8,"test":"C:/Users/USER/Desktop/jiwon/las-pytorch/data/kospeech/processed/test-clean.csv","vocab_size":1500},"model":{"listener":{"num_layers":3}}}
test_dataset = AudioDataset(params, "test")
test_loader = AudioDataLoader(test_dataset, num_workers=1).loader


tf_rate = 0.9
#loss = []
#ler = []
val_loss = []
val_ler = []
val_step = 0
for i, (data) in tqdm(enumerate(test_loader), total=len(test_loader), leave=False, desc="Test"):
    # print(
    #     f"Current Epoch: {epoch} | Epoch step: {epoch_step}/{len(train_loader)} Validating step: {val_step}/{len(dev_loader)}",
    #     end="\r",
    #     flush=True,
    # )

    inputs = data[1]["inputs"].cuda()
    labels = data[2]["targets"].cuda()

    batch_loss, batch_ler = batch_iterator(
        batch_data=inputs,
        batch_label=labels,
        las_model=las_model,
        optimizer=optimizer,
        tf_rate=tf_rate,
        is_training=False,
        max_label_len=30,
        label_smoothing=0.1,
        vocab_dict=test_dataset.char2idx,
    )
    if i % 100 == 0:
        torch.cuda.empty_cache()
    val_loss.append(batch_loss)
    val_ler.extend(batch_ler)
    val_step += 1 

val_loss = np.array([sum(val_loss) / len(val_loss)])
val_ler = np.array([sum(val_ler) / len(val_ler)])

print("test loss :", val_loss)
print("test ler :", val_ler)

Test:   0%|          | 0/3659 [00:00<?, ?it/s]

test loss : [2.1720921]
test ler : [0.30126758]


In [41]:
batch_data = np.load(path)
batch_label = label
tf_rate = 0.9
is_training = True

raw_pred_seq, _ = las_model(
        batch_data=batch_data, batch_label=batch_label, teacher_force_rate=tf_rate, is_training=is_training,
    )
pred_y = (torch.cat([torch.unsqueeze(each_y, 1) for each_y in raw_pred_seq], 1)[:, :max_label_len, :]).contiguous()

SyntaxError: EOL while scanning string literal (Temp/ipykernel_34048/704738897.py, line 16)

In [ ]:
if label_smoothing == 0.0 or not (is_training):
        pred_y = pred_y.permute(0, 2, 1)  # pred_y.contiguous().view(-1,output_class_dim)
        true_y = torch.max(batch_label, dim=2)[1][:, :max_label_len].contiguous()  # .view(-1)

        loss = criterion(pred_y, true_y)
        # variable -> numpy before sending into LER calculator
        batch_ler = LetterErrorRate(
            torch.max(pred_y.permute(0, 2, 1), dim=2)[1].cpu().numpy(),  # .reshape(current_batch_size,max_label_len),
            true_y.cpu().data.numpy(),
        )  # .reshape(current_batch_size,max_label_len), data)

    else:
        true_y = batch_label[:, :max_label_len, :].contiguous()
        true_y = true_y.type(torch.cuda.FloatTensor) if use_gpu else true_y.type(torch.FloatTensor)
        loss = label_smoothing_loss(pred_y, true_y, label_smoothing=label_smoothing)
        # batch_ler = [1.0]
        # print(true_y)
        # print("vs")
        # print(pred_y)
        batch_ler = LetterErrorRate(
            torch.max(pred_y, dim=2)[1].cpu().numpy(),  # .reshape(current_batch_size,max_label_len),
            torch.max(true_y, dim=2)[1].cpu().data.numpy(),
        )  # .reshape(current_batch_size,max_label_len), data)
        

In [ ]:
batch_loss = loss.cpu().data.numpy()

print(batch_loss, batch_ler)